# Environmental footprint of digital content consumption - LCI database creation

In [1]:
import pandas as pd
import numpy as np
import brightway2 as bw
import wurst
from pathlib import Path
import copy

# Basic set-up

In [2]:
BW_PROJECT = 'iri' # <- insert your project name here
bw.projects.set_current(BW_PROJECT)
ECOINVENT_DB = 'ecoinvent 3.8 cutoff'

In [3]:
# Run some functions needed to create the Digital Content Consumption database
%run _Functions_to_create_database.ipynb

# Create Digital Content Consumption database

Import ecoinvent and biosphere databases into list of dictionaries to facilitate filtering

In [4]:
ei_db = [ds.as_dict() for ds in bw.Database(ECOINVENT_DB)]
biosphere_db = [ef.as_dict() for ef in bw.Database('biosphere3')]

Import inventories from Excel

In [5]:
LCI_PATH = Path(r"..\data\LCI_Digital-Content-Consumption.xlsx")

DS_ICT = bw.ExcelImporter(LCI_PATH)
DS_ICT.apply_strategies()
DS_ICT.match_database(ECOINVENT_DB, fields=('name', 'reference product', 'unit', 'location'))
DS_ICT.match_database("biosphere3", fields=('name', 'unit', 'categories'))
DS_ICT.match_database(fields=('name', 'reference product', 'unit', 'location'))
DS_ICT.statistics()

Extracted 1 worksheets in 0.11 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.22 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
29 exchanges
0 unlinked exchanges

(5, 29, 0)

The *ExcelImporter* requires to define the *reference product* for exchanges. However, apparently the Ecoinvent database as imported in BW2 uses *product* instead of *reference product* for exchanges

In [6]:
# Change reference product to product for exchanges
DS_ICT = correct_reference_product_exchanges(DS_ICT)

Some inventories in the Excel file are provided as tables that can be imported as pandas dataframe. *create_inventories_from_df* function uses the pandas dataframes to create the inventories in BW2 format

In [7]:
# Import inventories for digital contents
LCI_DigitalContent = pd.read_excel(LCI_PATH, sheet_name='LCI_DigitalContent', skiprows=1).fillna(0)

# Improt data traffic range for each digital content
Data_Traffic = pd.read_excel(LCI_PATH, sheet_name='Data_Traffic', skiprows=1, index_col=0)

# Create inventories from dataframe:
DS_DigitalContent = create_inventories_from_df(LCI_DigitalContent, Data_Traffic)

In [8]:
# Import inventories for users type
LCI_Users = pd.read_excel(LCI_PATH, sheet_name='LCI_Users', skiprows=1).fillna(0)

# Create inventories from dataframe
DS_Users = create_inventories_from_df(LCI_Users, Data_Traffic)

Create regional datasets

In [9]:
# Database with only GLO inventories
GLO_db = []
GLO_db = DS_ICT.data + DS_DigitalContent + DS_Users

# Datasets that will be regionalized / Some inventories remain global
GLO_ds = ['market for data centers, internet access',
          'core network, internet access, data centers node']
ds_to_regionalize = [ds for ds in GLO_db if ds['name'] not in GLO_ds]

# Creating datasets for all the locations with an electricity market in ecoinvent (187) 
# would require substantial computational efforts. Consequently, we selected a number of 
# locations based on the GHG emission intensity of their electricity mix
new_locations = {# Regions
                 'BR': 'Brazil', 
                 'CA': 'Canada',
                 'CN': 'China',
                 'IN': 'India',
                 'RER': 'Europe',
                 'US': 'United States',
                 # Europe
                 'NO': 'Norway',
                 'CH': 'Switzerland',
                 'FR': 'France',
                 'DK': 'Denmark',
                 'ES': 'Spain',
                 'IT': 'Italy',
                 'DE': 'Germany',
                 'PL': 'Poland',
                 'GB': 'United Kingdom',
                 # Other countries
                 'ZA': 'South Africa',
                 'AU': 'Australia',
                 'RU': 'Russia'
                 }

# Create datasets for the new locations
DS_new_locs = create_regional_inventories(ds_to_regionalize, list(new_locations.keys()), ei_db)

Link exchanges internally and to the ecoinvent and biosphere databases (i.e., add the 'input' field)

In [10]:
DigitalContentConsumption_db = link_exchanges(GLO_db + DS_new_locs, ei_db, biosphere_db)

In [11]:
len(DigitalContentConsumption_db) # Number of datasets in the Digital Content Consumption database

914

Write the database

In [12]:
my_db_name = 'Digital Content Consumption'
if my_db_name in bw.databases:
    del bw.databases[my_db_name]
wurst.write_brightway2_database(DigitalContentConsumption_db, my_db_name)

914 datasets
7945 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/07/2022 16:01:57
  Finished: 12/07/2022 16:01:58
  Total time elapsed: 00:00:00
  CPU %: 86.40
  Memory %: 3.05
Created database: Digital Content Consumption


# Create prospective databases

Import prospective ecoinvent databases into a dictionary

In [19]:
premise_db_2030 = [ds.as_dict() for ds in bw.Database('ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2030 (premise v1.3.9)')]
premise_db_2040 = [ds.as_dict() for ds in bw.Database('ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2040 (premise v1.3.9)')]
premise_db_2050 = [ds.as_dict() for ds in bw.Database('ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2050 (premise v1.3.9)')]

Link exchanges to the new database (only for GLO datasets)

In [21]:
premise_db_2030_linked = link_exchanges(GLO_db, premise_db_2030, biosphere_db)
premise_db_2040_linked = link_exchanges(GLO_db, premise_db_2040, biosphere_db)
premise_db_2050_linked = link_exchanges(GLO_db, premise_db_2050, biosphere_db)

Write prospective databases

In [22]:
my_dbs_prospective = {'Digital Content Consumption, IMAGE SSP2-RCP19, 2030 (premise v1.3.9)': premise_db_2030_linked,
                      'Digital Content Consumption, IMAGE SSP2-RCP19, 2040 (premise v1.3.9)': premise_db_2040_linked,
                      'Digital Content Consumption, IMAGE SSP2-RCP19, 2050 (premise v1.3.9)': premise_db_2050_linked
                     }

for db in my_dbs_prospective:
    if db in bw.databases:
        del bw.databases[db]
    wurst.write_brightway2_database(my_dbs_prospective[db], db)

50 datasets
439 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/22/2022 16:24:11
  Finished: 11/22/2022 16:24:11
  Total time elapsed: 00:00:00
  CPU %: 78.10
  Memory %: 31.03
Created database: Digital Content Consumption, IMAGE SSP2-RCP19, 2030 (premise v1.3.9)
50 datasets
439 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/22/2022 16:24:13
  Finished: 11/22/2022 16:24:13
  Total time elapsed: 00:00:00
  CPU %: 40.10
  Memory %: 31.03
Created database: Digital Content Consumption, IMAGE SSP2-RCP19, 2040 (premise v1.3.9)
50 datasets
439 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/22/2022 16:24:14
  Finished: 11/22/2022 16:24:14
  Total time elapsed: 00:00:00
  CPU %: 24.80
  Memory %: 31.03
Created database: Digital Content Consumption, IMAGE SSP2-RCP19, 2050 (premise v1.3.9)
